# **Second Model**


# **Imports**

In [ ]:
%pip install pytorch_lightning -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 48.2 MB/s eta 0:00:00


In [ ]:
%pip install tensorboard -q

In [ ]:
import torch, torchvision
from torch import nn
from torch.nn import functional as F
from torchvision import models
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import random

In [ ]:
import pytorch_lightning as pylight
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Globals**

In [8]:
# defined number of batch size
BATCH_SIZE = 16
TEST_BATCH_SIZE = 20 # good to have different batch size if needed
# define max number of epochs
EPOCHS = 100
# defines on how many samples to run tests (may not be able to run for all)
MAX_TEST_ITER = 100 # batches # alternative: decrease test_set size

# **Utils**

## Auxiliary module





In [9]:
# A custom module used as replacement for vgg fc layers
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

## Triplet Loss

In [53]:
# Implementation of the loss required in the guidelines
class WeightedSoftMarginTripletLoss(nn.Module):
  def __init__(self, margin=8):
    super().__init__()
    self.margin = margin

  def forward(self, anchor, positive, negative):

    #print(anchor)
    #print(positive)
    #print(negative)

    first = torch.linalg.vector_norm(anchor - positive, dim=-1, keepdim=True)
    print(first)
    second = -torch.linalg.vector_norm(anchor - negative, dim=-1, keepdim=True)
    print(second)
    arg = self.margin * (second+first)
    const = torch.zeros((arg.shape[0],1)).to(device)
    arg = torch.cat((const,arg), dim=-1)
    #print(arg)
    return torch.logsumexp(arg, dim=-1).mean()


## Colour mapping

In [11]:
# Define color-to-class mapping for segmaps
# six available colours for six total classes
color_map = {
    (255, 255, 255): 0,   # white -> class 0
    (255, 0, 0): 1,       # red -> class 1
    (0, 0, 255): 2,       # blue -> class 2
    (0, 255, 0): 3,       # green -> class 3
    (255, 255, 0): 4,     # yellow -> class 4
    (0, 255, 255): 5,     # cyan -> class 5

    # pixels that are not in previous classes are considered 0
    (255, 0, 255): 0,
    (255, 255, 255): 0
}


def rgb_to_label(rgb_image):
    """Convert RGB image to class index tensor"""

    # Convert to numpy array
    np_image = np.array(rgb_image)
    # manage different colors
    np_image = np.where(np_image < 128, 0, 255)
    # Create empty label map
    label_map = np.zeros((np_image.shape[0], np_image.shape[1]), dtype=np.int64)

    # Map colors to class indices
    for color, class_idx in color_map.items():
        color_arr = np.array(color)
        mask = (np_image == color_arr).all(axis=-1)
        label_map[mask] = class_idx

    return torch.from_numpy(label_map).unsqueeze(0)

# **Data**
> WARNING: \
> If notebook is being tested paths and directories may need to be changed

## Dataset

In [12]:
# Dataset definition
class CVUSADataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, triplet_list, img_size=224, train=False, add_dir = ""):
        self.dataset_dir = dataset_dir
        self.triplet_list = triplet_list
        self.img_size = img_size
        self.add_dir = add_dir

        # transformations with data augmentation
        if train:
            # For ground view images
            self.ground_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(10),
                transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     # standard ImageNet normalization
            ])
        else:
            # For ground view images
            self.ground_transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     # standard ImageNet normalization
            ])


        # For aerial images
        self.aerial_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])     # standard ImageNet normalization
        ])

        # For aerial segmentation maps
        self.segmentation_transform = transforms.Compose([
            transforms.Resize((img_size, img_size), interpolation=transforms.InterpolationMode.NEAREST),
            transforms.Lambda(rgb_to_label),
            transforms.Lambda(lambda x: x.squeeze(0).long())  # (H, W) int64 tensor
        ])


    def __len__(self):
        return len(self.triplet_list)

    def __getitem__(self, idx):
        ground_rel, aerial_rel, seg_rel = self.triplet_list[idx]

        max_retries = 3
        for attempt in range(max_retries):
            try:
                # Load images
                ground_img = Image.open(self.dataset_dir + ground_rel).convert('RGB')
                aerial_img = Image.open(self.dataset_dir + aerial_rel).convert('RGB')

                # DISCLAIMER: Path to directories may need to change depending on runtime and user
                synth_img = Image.open((self.dataset_dir + "generated_images/generated_images_model_1/"+self.add_dir+"/segmap_predictions/seg_pred_"+ground_rel.split("/")[-1].split(".")[0]+".png").strip()).convert('RGB')
                seg_synth_img = Image.open((self.dataset_dir + "generated_images/generated_images_model_1/"+self.add_dir+"/aerial_predictions/aerial_pred_"+ground_rel.split("/")[-1].split(".")[0]+".png").strip()).convert('RGB')

                # Break if successful
                break
            except Exception as e:
                print(f"Error loading images for index {idx}: {e}")
                if attempt == max_retries - 1:
                    raise e


        # Apply transforms
        ground_tensor = self.ground_transform(ground_img).to(dtype=torch.float32)
        aerial_tensor = self.aerial_transform(aerial_img).to(dtype=torch.float32)
        synth_tensor = self.aerial_transform(synth_img).to(dtype=torch.float32)
        seg_synth_tensor = self.segmentation_transform(seg_synth_img).to(dtype=torch.float32)

                              # candidate    # generated   # segmented generated
        return ground_tensor, aerial_tensor, synth_tensor, seg_synth_tensor  # ground, aerial, synthetic_aerial, segmented_aerial

In [13]:
# Reads connection between images in different directories in which data is stored
def read_triplets_csv(csv_path):
    """Reads CSV file into list of (aerial, ground, seg) triplets"""
    triplets = []
    with open(csv_path, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            triplets.append((
                parts[0].strip(),  # streetview
                parts[1].strip(),  # polar aerial
                parts[2].strip()   # polar seg
            ))
    return triplets


dataset_dir = "/content/drive/MyDrive/CVUSA_subset/"

In [14]:
# DISCLAIMER: Path to directories may need to change depending on runtime and user
train_triplets = read_triplets_csv("/content/drive/MyDrive/CVUSA_subset/train.csv")
train_triplets, val_triplets = train_test_split(train_triplets, test_size=0.15, random_state=19)  # training/validation set
test_triplets = read_triplets_csv("/content/drive/MyDrive/CVUSA_subset/val.csv")        # test set

In [15]:
# Create Datasets
# DISCLAIMER: Path to directories may need to change depending on runtime and user
train_dataset = CVUSADataset(dataset_dir, train_triplets, train=True, add_dir="train_set")
val_dataset = CVUSADataset(dataset_dir, val_triplets, add_dir="val_set")
test_dataset = CVUSADataset(dataset_dir, test_triplets, add_dir="test_set")

# Create Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False,drop_last=True)

# **Network**

## Ground Branch
> for ground view processing

### simplified version

In [ ]:
class GroundBranchSim(nn.Module):
  def __init__(self, use_seg=False):
    super().__init__()

    self.use_seg = use_seg

    self.vgg1 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    self.vgg2 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

    # Freezing initial layers for finetuning
    for param1, param2 in zip(self.vgg1.features.parameters(), self.vgg2.features.parameters()):#, self.vgg1.features.parameters()):
      param1.requires_grad = False
      param2.requires_grad = False

    # If needed can modify size of final output...
    #self.vgg1.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    #self.vgg2.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

    # Modify size of input
    self.vgg1.features[0] = nn.Conv2d(
        1, # This may need to be 3...
        64,
        kernel_size=3,
        padding=1
    )
    # Initiate weights
    nn.init.kaiming_normal_(self.vgg1.features[0].weight)

    # Feed Forward Network turns output of VGG into embedding # TODO: decide final size...
    self.FNN = nn.Sequential(
      nn.Linear(1000, 1024),
      nn.LayerNorm(1024),
      nn.ReLU(),

      nn.Linear(1024, 512)
    )

  def forward(self, ground_view):
    x_ground = self.vgg2(ground_view)

    x = x_ground

    x = self.FNN(x)

    return x

### amplified version

In [30]:
class GroundBranch(nn.Module):
  def __init__(self, use_seg=False):
    super().__init__()

    self.use_seg = use_seg

    self.vgg1 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

    # Freezing first few conv layers for finetuning
    for n, param1 in enumerate(self.vgg1.features.parameters()):
      if n >=6:
        break
      param1.requires_grad = False

    # Remove fully connected layer, prioritize convolutions and pooling
    self.vgg1.classifier = nn.Sequential(Identity())

    self.GAP = nn.AdaptiveAvgPool2d((1,1))

    # Feed Forward Network turns output of VGG into embedding # TODO: decide final size...
    self.FNN = nn.Sequential(
      nn.Linear(512, 768),
      nn.LayerNorm(768),
      nn.ReLU(),

      nn.Linear(768, 512)
    )

  def forward(self, ground_view):
    x_ground = self.vgg1(ground_view).view(-1,512,7,7)

    return x_ground
    # Output still requires additional processing
    """
    x = self.GAP(x_ground)
    x = x.view(x.shape[0], -1)

    x = self.FNN(x)

    return x
    """

## Aerial Branch

### simplified version

In [ ]:
class AerialBranchSim(nn.Module):
  def __init__(self):
    super().__init__()

    self.vgg1 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    self.vgg2 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

    # Freezing initial layers for finetuning
    for param1, param2 in zip(self.vgg1.features.parameters(), self.vgg2.features.parameters()):
      param1.requires_grad = False
      param2.requires_grad = False

    # Modify size of input for segmaps which are 1 channel deep
    self.vgg1.features[0] = nn.Conv2d(
        1,
        64,
        kernel_size=3,
        padding=1
    )
    # Initiate weights
    nn.init.kaiming_normal_(self.vgg1.features[0].weight)

    # Feed Forward Network turns output of VGG into embedding
    self.FNN = nn.Sequential(
        nn.Linear(3000, 2048),
        nn.LayerNorm(2048),
        nn.ReLU(),

        nn.Linear(2048, 1024),
        nn.LayerNorm(1024),
        nn.ReLU(),

        nn.Linear(1024, 512)
    )

  def forward(self, synthetic_aerial, segmented_aerial, candidate_aerial):

    x_segmented = self.vgg1(segmented_aerial)
    x_synthetic = self.vgg2(synthetic_aerial)
    x_candidate = self.vgg2(candidate_aerial)

    x = torch.cat((x_synthetic, x_segmented, x_candidate), dim=-1)
    x = self.FNN(x)

    return x

### amplified version

In [29]:
class AerialBranch(nn.Module):
  def __init__(self):
    super().__init__()

    self.vgg1 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    self.vgg2 = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

    # Freezing initial layers for finetuning
    for n, (param1, param2) in enumerate(zip(self.vgg1.features.parameters(), self.vgg2.features.parameters())):
      if n >=6:
        break
      param1.requires_grad = False
      param2.requires_grad = False

    # Modify size of input
    self.vgg1.features[0] = nn.Conv2d(
        1,
        64,
        kernel_size=3,
        padding=1
    )
    # Initiate weights
    nn.init.kaiming_normal_(self.vgg1.features[0].weight)

    # Remove fully connected layer
    self.vgg1.classifier = nn.Sequential(Identity())
    self.vgg2.classifier = nn.Sequential(Identity())

    self.conv = nn.Conv2d(512,512,1)
    self.sigma = nn.Sigmoid()
    self.GAP = nn.AdaptiveAvgPool2d((1,1))

    # Feed Forward Network turns output of VGG into embedding # TODO: decide final size...
    self.FNN = nn.Sequential(
        nn.Linear(1536, 768),
        nn.LayerNorm(768),
        nn.ReLU(),

        nn.Linear(768, 512)
    )

  def forward(self, synthetic_aerial, segmented_aerial, candidate_aerial):

    if len(segmented_aerial.shape) == 3:
      segmented_aerial = segmented_aerial.unsqueeze(1)

    x_segmented = self.vgg1(segmented_aerial).view(-1,512,7,7)
    x_synthetic = self.vgg2(synthetic_aerial).view(-1,512,7,7)
    x_candidate = self.vgg2(candidate_aerial).view(-1,512,7,7)

    mask = self.conv(x_segmented)

    x_candidate *= mask
    x_synthetic *= mask

    x_candidate = self.GAP(x_candidate)
    x_candidate = x_candidate.view(x_candidate.shape[0], -1)
    x_synthetic = self.GAP(x_synthetic)
    x_synthetic = x_synthetic.view(x_synthetic.shape[0], -1)
    x_segmented = self.GAP(x_segmented)
    x_segmented = x_segmented.view(x_segmented.shape[0], -1)
    x = torch.cat((x_synthetic, x_segmented, x_candidate), dim=-1)
    x = self.FNN(x)

    return x

## Complete Network

In [31]:
# Just used to pull together the two branches
class CompNet(nn.Module):
  def __init__(self, ground_branch=GroundBranch(), aerial_branch=AerialBranch()):
    super().__init__()
    self.GB = ground_branch
    self.AB = aerial_branch

  def forward(self, ground_view, synthetic_aerial, segmented_aerial, candidate_aerial):
    return self.GB(ground_view), self.AB(synthetic_aerial, segmented_aerial, candidate_aerial)

# **Train**

## Lightning Wrapper

In [42]:
# Wrapper from lightning used to aid in training
class LightningWrapper(pylight.LightningModule):
  def __init__(self, device, model=CompNet(), batch_size=BATCH_SIZE):
    super().__init__()
    self.dvc=device

    self.model=model
    self.criterion=WeightedSoftMarginTripletLoss()

    # Non trainable parameters
    self.mask = torch.ones((batch_size,batch_size), requires_grad=False).to(self.dvc)-torch.eye(batch_size, requires_grad=False).to(self.dvc)

  def configure_optimizers(self):
    # very standard optimizers
    optimizer = torch.optim.AdamW(self.model.parameters(), lr=1e-3, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    return {"optimizer":optimizer, "lr_scheduler":scheduler}

  def forward(self, ground_view, synthetic_aerial, segmented_aerial, candidate_aerial):
    return self.model(ground_view, synthetic_aerial, segmented_aerial, candidate_aerial)

  def training_step(self, batch, batch_idx):
    ground, aerial, synthetic_aerial, segmented_aerial = batch

    ground = ground.to(self.dvc)
    aerial = aerial.to(self.dvc)
    synthetic_aerial = synthetic_aerial.to(self.dvc)
    segmented_aerial =  segmented_aerial.to(self.dvc)

    # Forward pass
    labels, predictions = self.model( ground, synthetic_aerial, segmented_aerial, aerial)

    # Computes cosine similarity of all ground embedding vectors in the batch before pooling
    with torch.no_grad():
      embs = labels.view(labels.shape[0],-1)
      embs = F.normalize(embs,dim = -1)
      embs = embs @ embs.T * self.mask

    # Completes processing of ground embeddings in forward pass (yes, in indsight all of this could've been added directly into the ground branch)
    labels = self.model.GB.GAP(labels)
    labels = labels.view(labels.shape[0],-1)
    labels = self.model.GB.FNN(labels)

    batch_size = len(predictions)

    # Find closest (hard) negative in batch by confronting ground view embeddings (a compromise due to computational constraints)
    closest_idx = torch.argmax(embs, dim=1)
    negatives = self.model.AB(synthetic_aerial,segmented_aerial,aerial[closest_idx,:,:])
    anchors = labels
    positives = predictions

    # Sanity check
    if negatives.shape[0] != batch_size:
      print("Not Sane")

    # compute triplet loss
    loss = self.criterion(anchors, positives, negatives)
    self.log("train_loss", loss, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    ground, aerial, synthetic_aerial, segmented_aerial = batch

    ground = ground.to(self.dvc)
    aerial = aerial.to(self.dvc)
    synthetic_aerial = synthetic_aerial.to(self.dvc)
    segmented_aerial =  segmented_aerial.to(self.dvc)

    # Forward pass
    labels, predictions = self.model( ground, synthetic_aerial, segmented_aerial, aerial)

    # Computes cosine similarity of all ground embedding vectors in the batch before pooling
    with torch.no_grad():
      embs = labels.view(labels.shape[0],-1)
      embs = F.normalize(embs,dim = -1)
      embs = embs @ embs.T * self.mask

    # Completes processing of ground embeddings in forward pass (yes, in indsight all of this could've been added directly into the ground branch)
    labels = self.model.GB.GAP(labels)
    labels = labels.view(labels.shape[0],-1)
    labels = self.model.GB.FNN(labels)

    batch_size = len(predictions)

    # Find closest (hard) negative in batch by confronting ground view embeddings (a compromise due to computational constraints)
    closest_idx = torch.argmax(embs, dim=1)
    print("debug in step")
    print(synthetic_aerial.shape, segmented_aerial.shape, aerial[closest_idx,:,:].shape)
    negatives = self.model.AB(synthetic_aerial,segmented_aerial,aerial[closest_idx,:,:])
    print(negatives.shape)
    print(negatives)
    anchors = labels
    print(labels)
    positives = predictions
    print(positives)
    print("debug in step over")
    # Sanity check
    if negatives.shape[0] != batch_size:
      print("Not Sane")

    # compute triplet loss
    loss = self.criterion(anchors, positives, negatives)
    self.log("val_loss", loss, prog_bar=True)
    return {"val_loss":loss}


## Lightning Training

In [ ]:
lightning_model = LightningWrapper(device).to(device)

# DISCLAIMER: Path to directories may need to change depending on runtime and user
log_path = "/content/drive/MyDrive/SavedModels/CV/test_attempt"
ckpt_path = log_path+"/checkpoints"

# Creates checkpoints periodically during training
checkpoint_callback = ModelCheckpoint(
    dirpath=ckpt_path,
    filename="best-checkpoint-{epoch:02d}-{val_loss:-2f}",
    save_top_k=3,
    verbose=True,
    monitor="val_loss",
    mode="min",
    save_last=True,
    every_n_epochs=2
)

# Performs training
trainer = Trainer(
    enable_checkpointing=True,
    default_root_dir=log_path,
    callbacks=[checkpoint_callback],
    enable_progress_bar=True,
    max_time = {"hours":10},
    accelerator="gpu",
    detect_anomaly=True,
    max_epochs = EPOCHS
)

# Training call
trainer.fit(lightning_model, train_dataloaders=train_loader, val_dataloaders=val_loader, ckpt_path=None)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                          | Params | Mode 
--------------------------------------------------------------------
0 | model     | CompNet                       | 46.8 M | eval 
1 | criterion | WeightedSoftMarginTripletLoss | 0      | train
--------------------------------------------------------------------
46.4 M    Trainable params
335 K     Non-trainable params
46.8 M    Total params
187.082   Total estimated model params size (MB)
1         Modules in train mode
125       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 2824: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 3530: 'val_loss' was not in top 3


## Visualization

In [ ]:
# Check model behavior during train
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/SavedModels/CV/test_attempt/lightning_logs

In [54]:
# This is for testing purposes only
# TODO: delete
lightning_model = LightningWrapper(device).eval()
w = WeightedSoftMarginTripletLoss()
with torch.no_grad():
  for n, (batch) in enumerate(train_loader):
    if n >= 1:
      break
    l = lightning_model.validation_step(batch, n)
    print(l)

debug in step
torch.Size([16, 3, 224, 224]) torch.Size([16, 224, 224]) torch.Size([16, 3, 224, 224])
torch.Size([16, 512])
tensor([[ 0.6732, -0.1168,  0.0227,  ..., -0.0734, -0.3013,  0.4642],
        [ 0.2142,  0.0888, -0.1341,  ...,  0.0464, -0.2372,  0.6586],
        [ 0.5402, -0.5149,  0.2021,  ..., -0.1384, -0.2855,  0.6619],
        ...,
        [ 0.1071, -0.0989, -0.0394,  ...,  0.1928, -0.0164,  0.2512],
        [-0.0071, -0.2436, -0.2622,  ...,  0.2631, -0.2453,  0.6333],
        [ 0.6821,  0.0884,  0.0548,  ...,  0.1051, -0.6261,  0.4283]])
tensor([[ 0.1550, -0.0918, -0.6219,  ..., -0.4098,  0.0880, -0.8315],
        [ 0.1748, -0.3164, -0.2922,  ..., -0.1682, -0.2577, -0.5174],
        [ 0.3094,  0.1980, -0.2515,  ..., -0.0598,  0.2933, -0.8474],
        ...,
        [-0.1979, -0.2357, -0.4371,  ..., -0.3298,  0.3370, -0.7012],
        [ 0.2209, -0.3672, -0.4596,  ..., -0.2842,  0.0945, -0.4672],
        [ 0.1356, -0.0210, -0.5912,  ...,  0.2091,  0.3258, -1.0196]])
tensor([[

# **Test**
> Testing discriminative model for every image in test set against every other one is unfeasible with our resources.\
 We instead propose the following approach.

In [ ]:
# Load trained model
ckpt_path = "..."
lightning_model = LightningWrapper.load_from_checkpoint("/path/to/checkpoint.ckpt")
# disable randomness, dropout, etc...
lightning_model.eval()

In [ ]:
iterations = len(test_loader)
print(iterations)
correct_guesses = []
for idx, batch in enumerate(test_loader):
  if idx >= MAX_TEST_ITER:
    break
  # For each element in batch select a random batch to confront
  id_list = range(iterations)
  id_list.remove(idx)
  random_ids = np.random.choice(id_list, size=TEST_BATCH_SIZE, replace=True)
  adversaries = test_loader[random_ids]

  ground, aerial, synthetic_aerial, segmented_aerial = batch

  ground = ground.to(device)
  aerial = aerial.to(device)
  synthetic_aerial = synthetic_aerial.to(device)
  segmented_aerial =  segmented_aerial.to(device)

  # Forward pass
  labels, predictions = lightning_model( ground, synthetic_aerial, segmented_aerial, aerial)

  # Completes processing of ground embeddings in forward pass
  labels = lightning_model.model.GB.GAP(labels)
  labels = labels.view(labels.shape[0],-1)
  labels = lightning_model.model.GB.FNN(labels)

  for n, pred in enumerate(predictions):
    _, fake_candidates, _, _ = adversaries[n]
    true_labels = labels[n].repeat(TEST_BATCH_SIZE,1)
    true_synth = synthetic_aerial[n].repeat(TEST_BATCH_SIZE,1,1,1)
    true_seg = segmented_aerial[n].repeat(TEST_BATCH_SIZE,1,1)

    fake_candidates_emb = lightning_model.model.AG(true_synth, true_seg, fake_candidate)

    fake_similarity = torch.min(F.mse_loss(true_labels, fake_candidates_emb))
    true_similarity = F.mse_loss(labels[n],predictions[n])

    # Could add some tolerance maybe (i.e. instead of being closest prediction is top 5)
    if true_similarity < fake_similarity:
      correct_guesses.append(1)
    else:
      correct_guesses.append(0)

accuracy = correct_guesses.sum()/len(correct_guesses)
print("Estimated accuracy", accuracy)